In [ ]:
from pyspark.sql.functions import when, col, udf
from pyspark.sql.types import StringType
# ensure the below library is installed on your fabric environment
import reverse_geocoder as rg


StatementMeta(, , , Waiting, , Waiting)

In [6]:

df = spark.read.table("earthquake_events_silver")#.filter(col('time') > start_date)

StatementMeta(, d384dd69-cc2f-43bd-a8d3-fd35cd01d54f, 10, Finished, Available, Finished)

In [7]:
display(df)

StatementMeta(, d384dd69-cc2f-43bd-a8d3-fd35cd01d54f, 11, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, f25ec3e5-361b-4c5a-83ce-7ce747a2f4a5)

In [8]:
def get_country_code(lat, lon):
    """
    Retrieve the country code for a given latitude and longitude.

    Parameters:
    lat (float or str): Latitude of the location.
    lon (float or str): Longitude of the location.

    Returns:
    str: Country code of the location, retrieved using the reverse geocoding API.

    Example:
    >>> get_country_details(48.8588443, 2.2943506)
    'FR'
    """
    coordinates = (float(lat), float(lon))
    return rg.search(coordinates)[0].get('cc')

StatementMeta(, d384dd69-cc2f-43bd-a8d3-fd35cd01d54f, 12, Finished, Available, Finished)

In [9]:
# registering the udfs so they can be used on spark dataframes
get_country_code_udf = udf(get_country_code, StringType())

StatementMeta(, d384dd69-cc2f-43bd-a8d3-fd35cd01d54f, 13, Finished, Available, Finished)

In [10]:
# adding country_code and city attributes
df_with_location = \
                df.\
                    withColumn("country_code", get_country_code_udf(col("latitude"), col("longitude")))


StatementMeta(, d384dd69-cc2f-43bd-a8d3-fd35cd01d54f, 14, Finished, Available, Finished)

In [11]:
# adding significance classification
df_with_location_sig_class = \
                            df_with_location.\
                                withColumn('sig_class', 
                                            when(col("sig") < 100, "Low").\
                                            when((col("sig") >= 100) & (col("sig") < 500), "Moderate").\
                                            otherwise("High")
                                            )

StatementMeta(, d384dd69-cc2f-43bd-a8d3-fd35cd01d54f, 15, Finished, Available, Finished)

In [12]:
# appending the data to the gold table
df_with_location_sig_class.write.mode('append').saveAsTable('earthquake_events_gold')

StatementMeta(, d384dd69-cc2f-43bd-a8d3-fd35cd01d54f, 16, Finished, Available, Finished)